# ML-Enhanced ASX Stock Predictor

This notebook demonstrates the **Machine Learning enhanced** stock prediction system that combines:

- **Deep Learning (LSTM)** with TensorFlow for sequence prediction
- **K-fold Cross-Validation** for robust model evaluation
- **10 Technical Analysis Methods** integrated as features
- **Hybrid Predictions** combining ML and traditional analysis
- **Backtesting** against historical data

## Features

- 🤖 **LSTM Neural Networks** for time series prediction
- 📊 **Feature Engineering** from technical indicators
- ✅ **K-fold Validation** with performance metrics
- 🔮 **Multi-horizon Predictions** (tomorrow, week, month)
- 📈 **Backtesting Framework** for validation
- 🎯 **Ensemble Predictions** (60% ML + 40% TA)

In [ ]:
# Import required libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime, timedelta
import yfinance as yf
import warnings
warnings.filterwarnings('ignore')

# Import the ML-enhanced predictor
from ml_asx_predictor import MLASXPredictor

# Set plotting style
plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette("husl")
%matplotlib inline

print("✓ Libraries imported successfully")

## 1. Load Stock Data

Test with one of the specified stocks or any ASX stock.

In [ ]:
# Configuration - Change to any ASX stock
TICKER = "FLT.AX"  # Flight Centre
PERIOD = "2y"      # Need more data for ML training

print(f"Downloading data for {TICKER}...")
df = yf.download(TICKER, period=PERIOD, progress=False)

print(f"\n✓ Data loaded successfully!")
print(f"Date range: {df.index[0].date()} to {df.index[-1].date()}")
print(f"Total periods: {len(df)}")
print(f"Current price: ${df['Close'].iloc[-1]:.2f}")

# Display recent prices
print(f"\nRecent prices:")
print(df.tail())

## 2. Initialize ML Predictor

In [ ]:
# Create ML-enhanced predictor
predictor = MLASXPredictor(df, enable_ml=True)

print(f"✓ ML Predictor initialized for {TICKER}")
print(f"  Technical indicators calculated")
print(f"  ML features prepared")
print(f"  Ready for training")

## 3. Train ML Models with K-Fold Cross-Validation

This trains LSTM models for each prediction horizon (tomorrow, week, month) using 5-fold cross-validation.

In [ ]:
# Train models (this may take a few minutes)
print("Training ML models with K-fold cross-validation...")
print("This may take 5-10 minutes depending on your hardware.\n")

training_results = predictor.train_ml_models(
    epochs=30,           # Number of training epochs per fold
    batch_size=32,       # Batch size for training
    validation_split=0.2 # Validation split within each fold
)

print("\n✓ Training complete!")

## 4. View Training Results

In [ ]:
# Display training metrics
print("="*70)
print("TRAINING RESULTS SUMMARY")
print("="*70)

for horizon, results in training_results.items():
    if 'metrics' in results:
        metrics = results['metrics']
        print(f"\n{horizon.upper()} Prediction:")
        print(f"  MAE: ${metrics['mae']:.2f} ± ${metrics['std_mae']:.2f}")
        print(f"  RMSE: ${metrics['rmse']:.2f}")
        print(f"  R² Score: {metrics['r2']:.4f}")
        print(f"  MAPE: {metrics['mape']:.2f}%")

## 5. Make Predictions

Generate predictions using:
- Pure ML predictions
- Pure Technical Analysis predictions  
- Hybrid predictions (combined)

In [ ]:
# Get ML predictions
ml_predictions = predictor.predict_with_ml()

# Get hybrid predictions (ML + TA)
hybrid_predictions = predictor.predict_prices_hybrid()

# Display predictions
current_price = hybrid_predictions['current']['price']

print("="*70)
print(f"PREDICTIONS FOR {TICKER}")
print("="*70)
print(f"\nCurrent Price: ${current_price:.2f}")
print(f"Date: {hybrid_predictions['current']['date'].date()}")

for horizon in ['tomorrow', 'week', 'month']:
    if horizon in hybrid_predictions:
        pred = hybrid_predictions[horizon]
        print(f"\n{horizon.upper()}:")
        print(f"  🎯 Hybrid Prediction: ${pred['prediction']:.2f} ({pred['change_percent']:+.2f}%)")
        print(f"  🤖 ML Only: ${pred['ml_prediction']:.2f}")
        print(f"  📊 TA Only: ${pred['ta_prediction']:.2f}")
        print(f"  📈 Range: ${pred['range_low']:.2f} - ${pred['range_high']:.2f}")
        print(f"  ✅ Confidence: {pred['confidence']:.0f}%")
        print(f"  ⚖️  Weights: {pred['ml_weight']:.0%} ML + {pred['ta_weight']:.0%} TA")

## 6. Visualize Predictions

In [ ]:
# Create comprehensive visualization
fig, axes = plt.subplots(2, 2, figsize=(16, 12))

# Plot 1: Price History with Predictions
ax1 = axes[0, 0]
recent_data = df.tail(100)
ax1.plot(recent_data.index, recent_data['Close'], label='Historical Price', linewidth=2, color='blue')

# Add prediction points
last_date = df.index[-1]
future_dates = [last_date + timedelta(days=1), last_date + timedelta(days=7), last_date + timedelta(days=30)]

# Hybrid predictions
hybrid_prices = [
    hybrid_predictions['tomorrow']['prediction'],
    hybrid_predictions['week']['prediction'],
    hybrid_predictions['month']['prediction']
]

# ML predictions
ml_prices = [
    hybrid_predictions['tomorrow']['ml_prediction'],
    hybrid_predictions['week']['ml_prediction'],
    hybrid_predictions['month']['ml_prediction']
]

# TA predictions
ta_prices = [
    hybrid_predictions['tomorrow']['ta_prediction'],
    hybrid_predictions['week']['ta_prediction'],
    hybrid_predictions['month']['ta_prediction']
]

ax1.scatter(future_dates, hybrid_prices, color='red', s=150, zorder=5, label='Hybrid Predictions', marker='o')
ax1.scatter(future_dates, ml_prices, color='green', s=100, zorder=5, label='ML Only', marker='^', alpha=0.6)
ax1.scatter(future_dates, ta_prices, color='orange', s=100, zorder=5, label='TA Only', marker='s', alpha=0.6)

ax1.set_title(f'{TICKER} - Price History and ML Predictions', fontsize=14, fontweight='bold')
ax1.set_xlabel('Date')
ax1.set_ylabel('Price ($)')
ax1.legend()
ax1.grid(True, alpha=0.3)

# Plot 2: Prediction Comparison
ax2 = axes[0, 1]
timeframes = ['Tomorrow', 'Week', 'Month']
hybrid_changes = [
    hybrid_predictions['tomorrow']['change_percent'],
    hybrid_predictions['week']['change_percent'],
    hybrid_predictions['month']['change_percent']
]

x = np.arange(len(timeframes))
width = 0.25

ml_changes = [(ml_prices[i] - current_price) / current_price * 100 for i in range(3)]
ta_changes = [(ta_prices[i] - current_price) / current_price * 100 for i in range(3)]

ax2.bar(x - width, ml_changes, width, label='ML', alpha=0.8, color='green')
ax2.bar(x, hybrid_changes, width, label='Hybrid', alpha=0.8, color='red')
ax2.bar(x + width, ta_changes, width, label='TA', alpha=0.8, color='orange')

ax2.axhline(y=0, color='black', linestyle='-', linewidth=0.5)
ax2.set_title('Prediction Comparison (%)', fontsize=14, fontweight='bold')
ax2.set_ylabel('Change (%)')
ax2.set_xticks(x)
ax2.set_xticklabels(timeframes)
ax2.legend()
ax2.grid(True, alpha=0.3, axis='y')

# Plot 3: Training Metrics
ax3 = axes[1, 0]
if training_results:
    horizons = list(training_results.keys())
    mapes = [training_results[h]['metrics']['mape'] for h in horizons if 'metrics' in training_results[h]]
    
    bars = ax3.bar(horizons, mapes, color=['green', 'yellow', 'orange'], alpha=0.7)
    ax3.set_title('Model Training MAPE (%)', fontsize=14, fontweight='bold')
    ax3.set_ylabel('MAPE (%)')
    ax3.grid(True, alpha=0.3, axis='y')
    
    for bar, mape in zip(bars, mapes):
        height = bar.get_height()
        ax3.text(bar.get_x() + bar.get_width()/2., height,
                f'{mape:.2f}%', ha='center', va='bottom')

# Plot 4: Confidence Scores
ax4 = axes[1, 1]
confidences = [
    hybrid_predictions['tomorrow']['confidence'],
    hybrid_predictions['week']['confidence'],
    hybrid_predictions['month']['confidence']
]
bars = ax4.bar(timeframes, confidences, color=['green', 'yellow', 'orange'], alpha=0.7)
ax4.set_title('Prediction Confidence Levels', fontsize=14, fontweight='bold')
ax4.set_ylabel('Confidence (%)')
ax4.set_ylim(0, 100)
ax4.grid(True, alpha=0.3, axis='y')

for bar, conf in zip(bars, confidences):
    height = bar.get_height()
    ax4.text(bar.get_x() + bar.get_width()/2., height,
            f'{conf:.0f}%', ha='center', va='bottom')

plt.tight_layout()
plt.show()

## 7. Backtest Predictions

Test how well the model would have performed on historical data.

In [ ]:
# Run backtest
print("Running backtest on last 20 days...")
backtest_results = predictor.backtest_predictions(test_days=20)

if 'metrics' in backtest_results:
    print("\n" + "="*70)
    print("BACKTEST RESULTS")
    print("="*70)
    
    for horizon, metrics in backtest_results['metrics'].items():
        print(f"\n{horizon.upper()}:")
        print(f"  Mean Absolute Error: ${metrics['mae']:.2f}")
        print(f"  RMSE: ${metrics['rmse']:.2f}")
        print(f"  MAPE: {metrics['mape']:.2f}%")
        print(f"  Max Error: {metrics['max_error']:.2f}%")
        print(f"  Predictions Tested: {metrics['n_predictions']}")

## 8. Comprehensive Analysis

Get full technical analysis combined with ML predictions.

In [ ]:
# Get comprehensive analysis
analysis = predictor.get_comprehensive_analysis_ml()

print("="*70)
print(f"COMPREHENSIVE ANALYSIS FOR {TICKER}")
print("="*70)

print(f"\n🎯 Overall Signal: {analysis['overall_signal']}")
print(f"   Bullish Signals: {analysis['bullish_signals']}")
print(f"   Bearish Signals: {analysis['bearish_signals']}")

print(f"\n📊 Key Indicators:")
ind = analysis['indicators']
print(f"   RSI: {ind['rsi']:.2f}")
print(f"   MACD: {ind['macd']:.4f}")
print(f"   ADX: {ind['adx']:.2f}")

print(f"\n📈 Trends:")
print(f"   Heikin Ashi: {analysis['heikin_ashi']['trend']}")
print(f"   Renko: {analysis['renko']['trend']}")

print(f"\n🤖 ML Enabled: {analysis['ml_enabled']}")

## 9. Test Multiple Stocks

Test the predictor on multiple stocks (this will take longer).

In [ ]:
# List of stocks to test
test_tickers = ['AEE.AX', 'BTL.AX', 'FLT.AX', 'IVV.AX']

print(f"Testing {len(test_tickers)} stocks (this may take 15-30 minutes)...\n")

multi_results = {}

for ticker in test_tickers:
    print(f"\n{'='*70}")
    print(f"Testing {ticker}")
    print('='*70)
    
    try:
        # Download data
        stock_df = yf.download(ticker, period='2y', progress=False)
        
        if len(stock_df) < 200:
            print(f"Skipping {ticker} - insufficient data")
            continue
        
        # Create and train predictor
        stock_predictor = MLASXPredictor(stock_df, enable_ml=True)
        stock_predictor.train_ml_models(epochs=20, batch_size=32)
        
        # Get predictions
        predictions = stock_predictor.predict_prices_hybrid()
        
        multi_results[ticker] = {
            'current': predictions['current']['price'],
            'tomorrow': predictions['tomorrow']['prediction'],
            'change_pct': predictions['tomorrow']['change_percent']
        }
        
        print(f"✓ {ticker}: ${predictions['current']['price']:.2f} → ${predictions['tomorrow']['prediction']:.2f} ({predictions['tomorrow']['change_percent']:+.2f}%)")
        
    except Exception as e:
        print(f"✗ Error with {ticker}: {str(e)}")

# Display summary
if multi_results:
    print(f"\n\n{'='*70}")
    print("MULTI-STOCK SUMMARY")
    print('='*70)
    
    summary_df = pd.DataFrame([
        {
            'Ticker': ticker,
            'Current': f"${data['current']:.2f}",
            'Tomorrow': f"${data['tomorrow']:.2f}",
            'Change %': f"{data['change_pct']:+.2f}%"
        }
        for ticker, data in multi_results.items()
    ])
    
    print(summary_df.to_string(index=False))

## Summary

This ML-enhanced predictor provides:

✅ **LSTM-based predictions** trained with K-fold validation  
✅ **Technical analysis integration** as feature engineering  
✅ **Hybrid predictions** combining ML and TA  
✅ **Backtesting framework** for validation  
✅ **Multi-horizon forecasts** (tomorrow, week, month)  
✅ **Comprehensive metrics** (MAE, RMSE, MAPE, R²)

## Disclaimer

⚠️ **For research and educational purposes only. Not financial advice.**

- Machine learning predictions are based on historical patterns
- Past performance does not guarantee future results
- Always conduct your own research
- Consider consulting a licensed financial advisor